## 利用神经网络实现minist手写数字分类

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
from sklearn.preprocessing import OneHotEncoder

In [2]:
data = loadmat('E://GitHub Code//Machine-Learning//03_神经网络//Dataset/ex4data1.mat')
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [3]:
X = data['X']       # (5000, 400)
y = data['y']       # (5000, 1)
X.shape, y.shape

((5000, 400), (5000, 1))

<font color=Sienna size=2>***one-hot 编码将类标签 n（k类）转换为长度为k的向量，其中索引 n 为“hot”（1），而其余为 0***</font>

In [4]:
encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(y)
y_onehot.shape      # (5000, 10) 每列第几个是1，表示属于第几类

(5000, 10)

In [5]:
y_onehot[::1000, :]     # array 切片，在行指定步长

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]])

In [6]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))


### 前向传播

(400 + 1) -> (25 + 1) -> (10)

&emsp;&emsp;![](https://img-blog.csdnimg.cn/20200925215436225.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80NTY3OTcwMA==,size_16,color_FFFFFF,t_70#pic_center)

&emsp;&emsp;![](https://img-blog.csdnimg.cn/20200928162523898.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80NTY3OTcwMA==,size_16,color_FFFFFF,t_70#pic_center)

In [7]:
def forward_propagate(X, theta1, theta2):
    rows = X.shape[0]       # 5000

    """
    在X的第0列，插入5000个1
    a1是输入层，具有401个神经元，也就是数据有多少个特征就有多少个神经元
    """
    a1 = np.insert(X, 0, values=np.ones(rows), axis=1)    # （5000， 401）

    """
    任务是分类，z2类似一个逻辑回归做的
    a2是第二层的神经元，25 + 1 个
    """
    # theta1是layer1 -> layer2的权重
    z2 = a1 * theta1.T     # (5000， 401) * (25, 401).T = (5000, 25)

    """
    a2是第二层的神经元
    sigmoid(z2): (5000, 25), 对第0列添加一列1
    a2: (5000, 26)
    """
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(rows), axis=1)

    """
    theta2: (10, 26)
    (5000, 26) * (10, 26) = (5000, 10)
    z3: (5000, 10)
    h: (5000, 10)
    """
    z3 = a2 * theta2.T
    h = sigmoid(z3)

    return a1, z2, a2, z3, h

In [8]:
def cost(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    rows = X.shape[0]
    X = np.mat(X)
    y = np.mat(y)

    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.mat(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.mat(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)

    # 计算每个样本X(i)的损失的累加和
    J = 0
    for i in range(rows):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)

    J = J / m

    return J

In [9]:
# 初始化设置
input_size = 400
hidden_size = 25
num_labels = 10
learning_rate = 1

# 随机初始化完整网络参数大小的参数数组
params = (np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.25

m = X.shape[0]
X = np.mat(X)
y = np.mat(y)

# 将参数数组解开为每个层的参数矩阵
theta1 = np.mat(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.mat(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

theta1.shape, theta2.shape

((25, 401), (10, 26))

In [10]:
a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
a1.shape, z2.shape, a2.shape, z3.shape, h.shape

((5000, 401), (5000, 25), (5000, 26), (5000, 10), (5000, 10))

In [11]:
cost(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate)

6.663170060595506

$(\frac{1}{1+e^{-z}})' = \frac{e^{-z}}{(1+e^{-z})^2} = sigmoid(z) \cdot (1 - sigomid(z))$<br>
$sigmoid(z) = \frac{1}{1+e^{-z}}$

In [12]:
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

$\xi^{L} = a^{L} - y$ &emsp;最后一层的输出 - 真实值<br>
$\xi^{(L-1)} = (\theta^{L-1})^T \xi^{L} * g'(z^{L-1})$

In [13]:
def backprop(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.mat(X)
    y = np.mat(y)

    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.mat(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.mat(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)

    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (10, 26)

    # compute the cost
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)

    J = J / m

    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))

    # perform backpropagation
    for t in range(m):
        a1t = a1[t,:]  # (1, 401)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        ht = h[t,:]  # (1, 10)
        yt = y[t,:]  # (1, 10)

        d3t = ht - yt  # (1, 10)

        z2t = np.insert(z2t, 0, values=np.ones(1))  # (1, 26)
        d2t = np.multiply((theta2.T * d3t.T).T, sigmoid_gradient(z2t))  # (1, 26)

        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t

    delta1 = delta1 / m
    delta2 = delta2 / m

    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))

    return J, grad

参数更新是用 $\theta:= \theta - \alpha \xi$ <font color=red>**???**</font>

In [14]:
from scipy.optimize import minimize

# minimize the objective function
fmin = minimize(fun=backprop, x0=params, args=(input_size, hidden_size, num_labels, X, y_onehot, learning_rate),
                method='TNC', jac=True, options={'maxiter': 250})
fmin

     fun: 0.5807531027139174
     jac: array([-0.00033413,  0.        ,  0.        , ..., -0.00273402,
       -0.00267997, -0.00058681])
 message: 'Max. number of function evaluations reached'
    nfev: 250
     nit: 18
  status: 3
 success: False
       x: array([-1.66680503,  0.11670953,  0.06366458, ...,  0.69904839,
        1.08082897, -0.81816384])

In [15]:
# 401 * 25 + 26 * 10 = 10285
fmin.x.shape

(10285,)

In [16]:
X = np.mat(X)
theta1 = np.mat(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.mat(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
y_pred = np.array(np.argmax(h, axis=1) + 1)
y_pred

array([[10],
       [10],
       [10],
       ...,
       [ 9],
       [ 9],
       [ 7]], dtype=int64)

In [17]:
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y)]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print ('accuracy = {0}%'.format(accuracy * 100))

accuracy = 96.2%
